In [1]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
import pandas as pd
df = pd.read_csv("../NBA_Play_Counts.csv", encoding='latin-1', index_col=0)
df = df[df['Total'] > 50].drop(columns=['Total']).sort_index()
df = df.div(df.sum(axis=1), axis=0) # Normalize by row
mean_vals = df.mean().rename("Average")
df = pd.concat([pd.DataFrame(mean_vals).transpose(), df])
df

,P&R Ball Handler,Spot-Up,Transition,ISO,Post-Up,P&R Roll Man,Cut,Offensive Rebound,Off Screen,Hand Off
Average,0.218985,0.203123,0.127733,0.056560,0.061319,0.067389,0.081569,0.106797,0.040615,0.035909
A.J. Price,0.656716,0.160448,0.059701,0.067164,0.000000,0.000000,0.003731,0.029851,0.003731,0.018657
AJ Hammons,0.000000,0.133333,0.033333,0.044444,0.222222,0.322222,0.077778,0.144444,0.011111,0.011111
AJ Price,0.575540,0.179856,0.071942,0.064748,0.000000,0.000000,0.021583,0.007194,0.000000,0.079137
Aaron Brooks,0.598434,0.109141,0.086806,0.114667,0.000000,0.000461,0.005296,0.030854,0.014736,0.039604
...,...,...,...,...,...,...,...,...,...,...
Zaza Pachulia,0.003121,0.085106,0.062695,0.047943,0.121702,0.138156,0.256454,0.277730,0.005106,0.001986
Zhaire Smith,0.132530,0.240964,0.204819,0.000000,0.000000,0.000000,0.084337,0.108434,0.060241,0.168675
Zhou Qi,0.019608,0.431373,0.137255,0.000000,0.039216,0.078431,0.078431,0.117647,0.039216,0.058824
Zion Williamson,0.028708,0.110048,0.167464,0.038278,0.196172,0.081340,0.167464,0.148325,0.023923,0.038278


In [3]:
def get_polar_plot(players):
    if not players:
        return px.line_polar(r = [0 for i in range(10)], theta = df.columns)
    a = pd.DataFrame({'Play Count': df.loc[players[0]], 'Player': players[0]})
    
    for p in players[1:]:
        b = pd.DataFrame({'Play Count': df.loc[p], 'Player': p}) 
        a = pd.concat([a, b])  
    
    a['Play Type'] = a.index
    return px.line_polar(a, r='Play Count', theta = 'Play Type', color='Player', line_close=True)

In [4]:
dropdown_options = [{"label":i, "value": i} for i in df.index]

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Play Type Distribution of Players"),
    html.P("Select any number of players from the dropdown below:"),
    dcc.Dropdown(id="selected_players", options=dropdown_options, value="Average", multi=True, placeholder="Search for a player or 'Average'",),
    dcc.Graph(id='play_type_dist')])

@app.callback(
    Output('play_type_dist', 'figure'),
    [Input("selected_players", "value")])
def update_figure(players):
    if isinstance(players, str):
        players = [players]
    return get_polar_plot(players)

app.run_server(mode="inline", debug=True)